In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import json
import psycopg2

if not os.path.exists(os.path.join(os.getcwd(), 'config.json')):
  # preventing to go always to the parent folder everytime we run the jupyter notebook without restarting
  os.chdir("..")
if not os.getcwd() in sys.path:
  sys.path.append(os.getcwd())

from labeler.database_labeler import label_database
from analysis.analysis_commons import clone_config_with_target, wikipedia_link_to_id, wikipedia_link_to_title

config = None
with open('config.json') as config_file:
  config = json.load(config_file)

conn = psycopg2.connect(**config["database"])
cur = conn.cursor()
cur.execute("SELECT id, url, content FROM nodes_info")
while True:
  node = cur.fetchone()
  if node is None:
      break
  
  node_id, node_url, content = node
  
  cur_inner = conn.cursor()
  cur_inner.execute("SELECT des FROM edges WHERE src = %s", (node_id,))
  edges = cur_inner.fetchall()
  outer_links = ", ".join(f"[[{edge[0]}]]" for edge in edges)
  
  markdown_content = f"# {node_url}\n\n---\n\n{content}\n\n---\n\n## Outer Links\n\n{outer_links}"
  
  with open(f"md_export/{node_id}.md", "w") as md_file:
      md_file.write(markdown_content)
  
  cur_inner.close()

cur.close()
conn.close()
